# Data Preparation - Cleaning, Wrangling, Deriving Variables, Merging, Integration

### Contents:

1. Importing Libraries and Data
2. Cleaning and Wrangling Hospital Readmissions Reductions Program Dataset
   - Deriving New Variables - Hospital Readmissions Reductions Program Dataset
3. Cleaning and Wrangling Hospital General Information Dataset
4. Cleaning and Wrangling HVBP Clinical Outcomes Dataset
   - Deriving New Variables - HVBP Clinical Outcomes Dataset
5. Data Merging
6. Consistency Checks on Integreated Dataset
7. Exporting 

# 01. Importing Libraries and Data

In [1]:
# Import libraries

import os
import pandas as pd
import numpy as np

In [2]:
# Create path

path = r'C:\Users\jenci\Documents\Data Analytics\Data Analytics\Advanced Analytics and Dashboard Design\Acheivement 6'

In [3]:
# Import dataframes

df_hrrp = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'hospital_readmissions_reduction_program.csv'), index_col=False)
df_hgi = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'hospital_general_information.csv'), index_col=False)
df_hvbp = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'hvbp_clinical_outcomes.csv'), index_col=False)

# 02. Cleaning and Wrangling Hospital Readmisssions Reductions Program Dataset

In [4]:
# Check shape

df_hrrp.shape

(18774, 12)

In [5]:
# Check dataframe

df_hrrp.head(5)

,Facility Name,Facility ID,State,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,NaN,NaN,0.8916,3.5325,3.9618,Too Few to Report,07/01/2019,06/30/2022
1,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HF-HRRP,616.0,NaN,1.1003,23.1263,21.0184,149,07/01/2019,06/30/2022
2,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-AMI-HRRP,274.0,NaN,0.9332,12.9044,13.8283,32,07/01/2019,06/30/2022
3,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-PN-HRRP,404.0,NaN,0.9871,17.0529,17.2762,68,07/01/2019,06/30/2022
4,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-CABG-HRRP,126.0,NaN,0.9517,9.8131,10.3112,11,07/01/2019,06/30/2022


> **Addressing Duplicates:**
> 
> During the data profiling phase, **verified the absence of duplicates.**

In [6]:
# Remove all rows that have Nulls in 'Predicted Readmissions Ratio' column and any values in the Footnote column

df_hrrp_copy = df_hrrp[df_hrrp['Predicted Readmission Rate'].notna()]

> Removing missing rows in the **'Predicted Readmission Rate'** column is based on the criterion that all subsequent quantitative variables related to it are also missing. These values are missing completely at random, primarily because hospitals did not participate in data submission despite being listed.

In [7]:
# Check output 
df_hrrp_copy['Predicted Readmission Rate'].isnull().sum()

0

> A sum of zero means all of the rows with missing data in **'Predicted Readmission Rate'** were removed.

In [8]:
# Check output

df_hrrp_copy.shape

(12077, 12)

In [9]:
# Remove 'Footnote' 'Start Date' 'End Date' 'Number of Discharges' 'Number of Readmissions; columns

df_hrrp_copy = df_hrrp_copy.drop(['Footnote', 'Start Date', 'End Date', 'Number of Discharges', 'Number of Readmissions'], axis=1)

> The decision is to remove **'Number of Discharges'** and **'Number of Readmissions'** from the dataset.
> 
> These columns contain a significant amount of missing data because the values were suppressed to protect patient privacy. Despite their importance as measurements, the data is incomplete due to this suppression.
> 
> By removing 'Number of Discharges' and 'Number of Readmissions', focus can be directed towards analyzing more complete and relevant data points, ensuring the analysis remains robust and actionable.
>
> Reasons for Dropping the Columns:
>
> 1. **Data Completeness**: Both 'Number of Discharges' and 'Number of Readmissions' have substantial missing values, which can impact the reliability and comprehensiveness of the analysis.
>
> 2. **Alternative Measures**: Fortunately, access remains available to other valuable metrics such as ratios and rates, which provide meaningful insights into hospital performance without compromising patient privacy.
>
> 3. **Reason for Missing Data**: The missing values in these columns are not due to a systematic issue but rather because hospitals chose not to participate in data submission for various reasons. This missingness can be considered at random and unrelated to specific hospital characteristics.

In [10]:
# Check output

df_hrrp_copy.head(1)

,Facility Name,Facility ID,State,Measure Name,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,0.8916,3.5325,3.9618


In [11]:
# Rename 'Predeicted Readmission Rate' to 'Observed Readmission Rate' -- Renaming this to clarify for general audiences

df_hrrp_copy.rename(columns={'Predicted Readmission Rate': 'Observed Readmission Rate'}, inplace=True)

In [12]:
# Check datatypes

df_hrrp_copy.dtypes

Facility Name                 object
Facility ID                    int64
State                         object
Measure Name                  object
Excess Readmission Ratio     float64
Observed Readmission Rate    float64
Expected Readmission Rate    float64
dtype: object

In [13]:
# Check for mixed data types

for col in df_hrrp_copy.columns.tolist():
    weird = (df_hrrp_copy[[col]].map(type) != df_hrrp_copy[[col]].iloc[0].apply(type)).any(axis=1)
    if len (df_hrrp_copy[weird]) > 0:
        print (col)

> Lack of output means there are **no mixed data types present in df_hrrp_copy.**

In [14]:
# Check for missing values

df_hrrp_copy.isnull().sum()

Facility Name                0
Facility ID                  0
State                        0
Measure Name                 0
Excess Readmission Ratio     0
Observed Readmission Rate    0
Expected Readmission Rate    0
dtype: int64

In [15]:
# Check shape

df_hrrp_copy.shape

(12077, 7)

## 02a. Deriving New Variables - Hospital Readmissions Reductions Program Dataset

In [16]:
# Define threhold for new flag
threshold_1 = 1.0

# Derive new variable called 'Excess Readmission Flag' 
df_hrrp_copy['Excess Readmissions Threshold'] = df_hrrp_copy['Excess Readmission Ratio'].apply(lambda x: 'Above Threshold' if x > threshold_1 else 'Below Threshold')

In [17]:
# Check output

df_hrrp_copy.head(1)

,Facility Name,Facility ID,State,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,0.8916,3.5325,3.9618,Below Threshold


In [18]:
# Check shape of cleaned and wrangled df_hrrp_copy dataframe

df_hrrp_copy.shape

(12077, 8)

# 03. Cleaning and Wrangling Hospital General Information Dataset

In [19]:
# Check shape

df_hgi.shape

(5425, 39)

In [20]:
# Check dataframe

pd.set_option('display.max_columns', None) # Set option to view all columns
df_hgi.head(5)

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Meets criteria for promoting interoperability of EHRs,Meets criteria for birthing friendly designation,Hospital overall rating,Hospital overall rating footnote,MORT Group Measure Count,Count of Facility MORT Measures,Count of MORT Measures Better,Count of MORT Measures No Different,Count of MORT Measures Worse,MORT Group Footnote,Safety Group Measure Count,Count of Facility Safety Measures,Count of Safety Measures Better,Count of Safety Measures No Different,Count of Safety Measures Worse,Safety Group Footnote,READM Group Measure Count,Count of Facility READM Measures,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,NaN,7,7,1,5,1,NaN,8,8,2,6,0,NaN,11,11,0,10,1,NaN,8,8,NaN,12,10,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,NaN,2,NaN,7,6,0,4,2,NaN,8,7,0,6,1,NaN,11,10,0,10,0,NaN,8,8,NaN,12,11,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 629-1000,Acute Care Hospitals,Proprietary,Yes,Y,Y,1,NaN,7,7,0,5,2,NaN,8,7,2,5,0,NaN,11,9,0,7,2,NaN,8,8,NaN,12,11,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Y,NaN,2,NaN,7,3,0,3,0,NaN,8,2,0,2,0,NaN,11,6,0,5,1,NaN,8,8,NaN,12,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,Yes,Y,NaN,Not Available,16.0,7,1,0,1,0,NaN,8,Not Available,Not Available,Not Available,Not Available,5.0,11,2,0,2,0,NaN,8,Not Available,5.0,12,6,NaN


> **Addressing Duplicates:**
> 
> During the data profiling phase, **verified the absence of duplicates.**

In [21]:
# Create copy of Hospital General Information dataframe
df_hgi_copy = df_hgi.copy()

# Create list of columns to remove
columns_to_remove_1 = [
    'Facility ID',
    'Hospital overall rating footnote',
    'MORT Group Measure Count',
    'Count of MORT Measures No Different',
    'MORT Group Footnote',
    'Safety Group Measure Count',
    'Count of Safety Measures No Different',
    'Safety Group Footnote',
    'READM Group Measure Count',
    'Count of READM Measures No Different',
    'READM Group Footnote',
    'Pt Exp Group Measure Count',
    'Pt Exp Group Footnote',
    'TE Group Measure Count',
    'Count of Facility Pt Exp Measures',
    'Count of Facility TE Measures',
    'TE Group Footnote'
]

# Remove list of columns from df_hgi_copy
df_hgi_copy = df_hgi_copy.drop(columns=columns_to_remove_1)

> **Opt to exclude 'Facility ID'** due to its differing data type, which complicates merging and isn't essential enough to justify the effort required for alignment, as there are several additional keys available for merging.

In [22]:
# Remove all rows that contain 'Not Avaialable' in 'Hospital Rating' columns

df_hgi_copy = df_hgi_copy[~df_hgi_copy['Hospital overall rating'].astype(str).str.contains('Not Available')]

> **Deleting rows with 'Not Available' entries in the 'Hospital overall rating' column is justified because** this variable stands out due to **its** unconnected nature from other variables of interest in our dataset, such as readmission rates and hospital characteristics. This **lack of interdependence ensures  that removing these rows does not compromise the integrity or interconnectedness of our analysis.**
>
> Additionally, the missing values in the 'Hospital overall rating' column are random and stem from various factors including CMS data standardization, insufficient rating counts, non-submission by hospitals, and discrepancies between submitted and CMS data. This randomness makes imputation less reliable and introduces potential biases. **Despite losing approximately 43% of the original data, this approach strengthens the validity of our analysis, allowing for a more focused evaluation of hospital performance characteristics.**


In [23]:
# Check shape

df_hgi_copy.shape

(3042, 22)

In [24]:
# Check for remaining null values

df_hgi_copy.eq('Not Available').sum()

Facility Name                                             0
Address                                                   0
City/Town                                                 0
State                                                     0
ZIP Code                                                  0
County/Parish                                             0
Telephone Number                                          0
Hospital Type                                             0
Hospital Ownership                                        0
Emergency Services                                        0
Meets criteria for promoting interoperability of EHRs     0
Meets criteria for birthing friendly designation          0
Hospital overall rating                                   0
Count of Facility MORT Measures                          65
Count of MORT Measures Better                            65
Count of MORT Measures Worse                             65
Count of Facility Safety Measures       

In [25]:
# Check datatypes of each columns -- In preparation to calculate mean as we need values to be numeric

df_hgi_copy.dtypes

Facility Name                                            object
Address                                                  object
City/Town                                                object
State                                                    object
ZIP Code                                                  int64
County/Parish                                            object
Telephone Number                                         object
Hospital Type                                            object
Hospital Ownership                                       object
Emergency Services                                       object
Meets criteria for promoting interoperability of EHRs    object
Meets criteria for birthing friendly designation         object
Hospital overall rating                                  object
Count of Facility MORT Measures                          object
Count of MORT Measures Better                            object
Count of MORT Measures Worse            

In [26]:
# Create list of columns in df_hgi_copy for which we need to change datatypes
columns_to_convert_2 = [
    'Count of Facility MORT Measures',
    'Count of MORT Measures Better',
    'Count of MORT Measures Worse',
    'Count of Facility Safety Measures',
    'Count of Safety Measures Better',
    'Count of Safety Measures Worse',
    'Count of Facility READM Measures',
    'Count of READM Measures Better',
    'Count of READM Measures Worse',
]

# Change datatypes to numeric
df_hgi_copy[columns_to_convert_2] = df_hgi_copy[columns_to_convert_2].apply(pd.to_numeric, errors='coerce').astype('float64')


In [27]:
# Check output

df_hgi_copy.dtypes

Facility Name                                             object
Address                                                   object
City/Town                                                 object
State                                                     object
ZIP Code                                                   int64
County/Parish                                             object
Telephone Number                                          object
Hospital Type                                             object
Hospital Ownership                                        object
Emergency Services                                        object
Meets criteria for promoting interoperability of EHRs     object
Meets criteria for birthing friendly designation          object
Hospital overall rating                                   object
Count of Facility MORT Measures                          float64
Count of MORT Measures Better                            float64
Count of MORT Measures Wo

In [28]:
# Replace 'Not Avaialble' with NaN in numeric columns
df_hgi_copy.replace('Not Available', np.nan, inplace=True)

# Create df with column means
column_means_2 = df_hgi_copy[columns_to_convert_2].mean().round()

# Fill NaN values in numeric columns with values in column_means_2
df_hgi_copy[columns_to_convert_2] = df_hgi_copy[columns_to_convert_2].fillna(column_means_2)

> For **imputing values, 'Not Available' is converted to NaN before calculating the mean of each numeric column.** Additionally, we are rounding because all the numbers are counts, and therefore, whole numbers.

In [29]:
# Check output

df_hgi_copy.eq('Not Available').sum()

Facility Name                                            0
Address                                                  0
City/Town                                                0
State                                                    0
ZIP Code                                                 0
County/Parish                                            0
Telephone Number                                         0
Hospital Type                                            0
Hospital Ownership                                       0
Emergency Services                                       0
Meets criteria for promoting interoperability of EHRs    0
Meets criteria for birthing friendly designation         0
Hospital overall rating                                  0
Count of Facility MORT Measures                          0
Count of MORT Measures Better                            0
Count of MORT Measures Worse                             0
Count of Facility Safety Measures                       

In [30]:
# Create list identifying columns for imputation with 'N'
columns_to_replace_1 = ['Meets criteria for promoting interoperability of EHRs', 'Meets criteria for birthing friendly designation']

# Impute missing values in these columns with 'N'
df_hgi_copy[columns_to_replace_1] = df_hgi_copy[columns_to_replace_1].fillna('N')

> **Imputing these values is necessary because the only unique variables present in the column are 'Y' for meeting the criteria and missing values.** Therefore, all blanks will be imputed with 'N' to signify non-compliance with the criteria.

In [31]:
# Check output

df_hgi_copy[columns_to_replace_1].value_counts()

Meets criteria for promoting interoperability of EHRs  Meets criteria for birthing friendly designation
Y                                                      Y                                                   1949
                                                       N                                                    771
N                                                      N                                                    206
                                                       Y                                                    116
Name: count, dtype: int64

In [32]:
# Check for mixed data types

for col in df_hgi_copy.columns.tolist():
    weird = (df_hgi_copy[[col]].map(type) != df_hgi_copy[[col]].iloc[0].apply(type)).any(axis=1)
    if len (df_hgi_copy[weird]) > 0:
        print (col)

> Absence of output means there are **no mixed data types present in df_hgi_copy.**

In [33]:
# Check for null values

df_hgi_copy.isnull().sum()

Facility Name                                            0
Address                                                  0
City/Town                                                0
State                                                    0
ZIP Code                                                 0
County/Parish                                            0
Telephone Number                                         0
Hospital Type                                            0
Hospital Ownership                                       0
Emergency Services                                       0
Meets criteria for promoting interoperability of EHRs    0
Meets criteria for birthing friendly designation         0
Hospital overall rating                                  0
Count of Facility MORT Measures                          0
Count of MORT Measures Better                            0
Count of MORT Measures Worse                             0
Count of Facility Safety Measures                       

In [34]:
# Rename columns

df_hgi_copy.rename(columns={
    'Meets criteria for promoting interoperability of EHRs': 'Promotes Interoperability of EHRs',
    'Meets criteria for birthing friendly designation': 'Birthing Friendly Facility',
    'Hospital overall rating': 'Hospital Overall Rating',
    'Count of MORT Measures Better': 'MORT Measures Better',
    'Count of MORT Measures Worse': 'MORT Measures Worse',
    'Count of Safety Measures Better': 'Safety Measures Better',
    'Count of Safety Measures Worse': 'Safety Measures Worse',
    'Count of READM Measures Better': 'READM Measures Better',
    'Count of READM Measures Worse': 'READM Measures Worse'
}, inplace=True)
    

In [35]:
# Check output

df_hgi_copy.head(1)

,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse
0,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0


In [36]:
# Check shape

df_hgi_copy.shape

(3042, 22)

# 04. Cleaning and Wrangling HVBP Clinical Outcomes Dataset

In [37]:
# Check shape

df_hvbp.shape

(2474, 50)

In [38]:
# Check HVBP Clinical Outcomes dataframe

df_hvbp.head(5)

,Fiscal Year,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,MORT-30-AMI Achievement Threshold,MORT-30-AMI Benchmark,MORT-30-AMI Baseline Rate,MORT-30-AMI Performance Rate,MORT-30-AMI Achievement Points,MORT-30-AMI Improvement Points,MORT-30-AMI Measure Score,MORT-30-HF Achievement Threshold,MORT-30-HF Benchmark,MORT-30-HF Baseline Rate,MORT-30-HF Performance Rate,MORT-30-HF Achievement Points,MORT-30-HF Improvement Points,MORT-30-HF Measure Score,MORT-30-PN Achievement Threshold,MORT-30-PN Benchmark,MORT-30-PN Baseline Rate,MORT-30-PN Performance Rate,MORT-30-PN Achievement Points,MORT-30-PN Improvement Points,MORT-30-PN Measure Score,MORT-30-COPD Achievement Threshold,MORT-30-COPD Benchmark,MORT-30-COPD Baseline Rate,MORT-30-COPD Performance Rate,MORT-30-COPD Achievement Points,MORT-30-COPD Improvement Points,MORT-30-COPD Measure Score,MORT-30-CABG Achievement Threshold,MORT-30-CABG Benchmark,MORT-30-CABG Baseline Rate,MORT-30-CABG Performance Rate,MORT-30-CABG Achievement Points,MORT-30-CABG Improvement Points,MORT-30-CABG Measure Score,COMP-HIP-KNEE Achievement Threshold,COMP-HIP-KNEE Benchmark,COMP-HIP-KNEE Baseline Rate,COMP-HIP-KNEE Performance Rate,COMP-HIP-KNEE Achievement Points,COMP-HIP-KNEE Improvement Points,COMP-HIP-KNEE Measure Score
0,2024,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,0.869247,0.887868,0.856752,0.880237,6 out of 10,7 out of 9,7 out of 10,0.882308,0.907733,0.872117,0.901917,7 out of 10,8 out of 9,8 out of 10,0.840281,0.872976,0.833710,0.815392,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.905159,0.909860,0 out of 10,1 out of 9,1 out of 10,0.969499,0.980319,0.954926,0.958983,0 out of 10,1 out of 9,1 out of 10,0.025396,0.018159,0.030343,0.022314,4 out of 10,6 out of 9,6 out of 10
1,2024,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,0.869247,0.887868,0.848053,0.864067,0 out of 10,4 out of 9,4 out of 10,0.882308,0.907733,0.854099,0.856339,0 out of 10,0 out of 9,0 out of 10,0.840281,0.872976,0.789256,0.765414,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.892007,0.898823,0 out of 10,1 out of 9,1 out of 10,0.969499,0.980319,Not Available,Not Available,Not Available,Not Available,Not Available,0.025396,0.018159,0.030034,0.019253,8 out of 10,9 out of 9,9 out of 10
2,2024,10006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,0.869247,0.887868,0.853581,0.835481,0 out of 10,0 out of 9,0 out of 10,0.882308,0.907733,0.872053,0.866923,0 out of 10,0 out of 9,0 out of 10,0.840281,0.872976,0.824859,0.805453,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.908992,0.899168,0 out of 10,0 out of 9,0 out of 10,0.969499,0.980319,0.967735,0.963606,0 out of 10,0 out of 9,0 out of 10,0.025396,0.018159,0.035949,0.037582,0 out of 10,0 out of 9,0 out of 10
3,2024,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,0.869247,0.887868,Not Available,Not Available,Not Available,Not Available,Not Available,0.882308,0.907733,0.865767,0.874346,0 out of 10,2 out of 9,2 out of 10,0.840281,0.872976,0.780354,0.717560,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.917078,0.865260,0 out of 10,0 out of 9,0 out of 10,0.969499,0.980319,Not Available,Not Available,Not Available,Not Available,Not Available,0.025396,0.018159,0.027836,Not Available,Not Available,Not Available,Not Available
4,2024,10011,ST. VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,JEFFERSON,0.869247,0.887868,0.859425,0.868485,0 out of 10,3 out of 9,3 out of 10,0.882308,0.907733,0.872458,0.858843,0 out of 10,0 out of 9,0 out of 10,0.840281,0.872976,0.830560,0.782196,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.905170,0.894272,0 out of 10,0 out of 9,0 out of 10,0.969499,0.980319,0.962992,0.973731,4 out of 10,6 out of 9,6 out of 10,0.025396,0.018159,0.026092,0.026132,0 out of 10,0 out of 9,0 out of 10


> **Addressing Duplicates:**
> 
> During the data profiling phase, **verified the absence of duplicates.**

In [39]:
# Create copy of Hospital General Information dataframe

df_hvbp_copy = df_hvbp.copy()

In [40]:
# Check output

df_hvbp_copy.head(5)

,Fiscal Year,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,MORT-30-AMI Achievement Threshold,MORT-30-AMI Benchmark,MORT-30-AMI Baseline Rate,MORT-30-AMI Performance Rate,MORT-30-AMI Achievement Points,MORT-30-AMI Improvement Points,MORT-30-AMI Measure Score,MORT-30-HF Achievement Threshold,MORT-30-HF Benchmark,MORT-30-HF Baseline Rate,MORT-30-HF Performance Rate,MORT-30-HF Achievement Points,MORT-30-HF Improvement Points,MORT-30-HF Measure Score,MORT-30-PN Achievement Threshold,MORT-30-PN Benchmark,MORT-30-PN Baseline Rate,MORT-30-PN Performance Rate,MORT-30-PN Achievement Points,MORT-30-PN Improvement Points,MORT-30-PN Measure Score,MORT-30-COPD Achievement Threshold,MORT-30-COPD Benchmark,MORT-30-COPD Baseline Rate,MORT-30-COPD Performance Rate,MORT-30-COPD Achievement Points,MORT-30-COPD Improvement Points,MORT-30-COPD Measure Score,MORT-30-CABG Achievement Threshold,MORT-30-CABG Benchmark,MORT-30-CABG Baseline Rate,MORT-30-CABG Performance Rate,MORT-30-CABG Achievement Points,MORT-30-CABG Improvement Points,MORT-30-CABG Measure Score,COMP-HIP-KNEE Achievement Threshold,COMP-HIP-KNEE Benchmark,COMP-HIP-KNEE Baseline Rate,COMP-HIP-KNEE Performance Rate,COMP-HIP-KNEE Achievement Points,COMP-HIP-KNEE Improvement Points,COMP-HIP-KNEE Measure Score
0,2024,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,0.869247,0.887868,0.856752,0.880237,6 out of 10,7 out of 9,7 out of 10,0.882308,0.907733,0.872117,0.901917,7 out of 10,8 out of 9,8 out of 10,0.840281,0.872976,0.833710,0.815392,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.905159,0.909860,0 out of 10,1 out of 9,1 out of 10,0.969499,0.980319,0.954926,0.958983,0 out of 10,1 out of 9,1 out of 10,0.025396,0.018159,0.030343,0.022314,4 out of 10,6 out of 9,6 out of 10
1,2024,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,0.869247,0.887868,0.848053,0.864067,0 out of 10,4 out of 9,4 out of 10,0.882308,0.907733,0.854099,0.856339,0 out of 10,0 out of 9,0 out of 10,0.840281,0.872976,0.789256,0.765414,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.892007,0.898823,0 out of 10,1 out of 9,1 out of 10,0.969499,0.980319,Not Available,Not Available,Not Available,Not Available,Not Available,0.025396,0.018159,0.030034,0.019253,8 out of 10,9 out of 9,9 out of 10
2,2024,10006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,0.869247,0.887868,0.853581,0.835481,0 out of 10,0 out of 9,0 out of 10,0.882308,0.907733,0.872053,0.866923,0 out of 10,0 out of 9,0 out of 10,0.840281,0.872976,0.824859,0.805453,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.908992,0.899168,0 out of 10,0 out of 9,0 out of 10,0.969499,0.980319,0.967735,0.963606,0 out of 10,0 out of 9,0 out of 10,0.025396,0.018159,0.035949,0.037582,0 out of 10,0 out of 9,0 out of 10
3,2024,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,0.869247,0.887868,Not Available,Not Available,Not Available,Not Available,Not Available,0.882308,0.907733,0.865767,0.874346,0 out of 10,2 out of 9,2 out of 10,0.840281,0.872976,0.780354,0.717560,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.917078,0.865260,0 out of 10,0 out of 9,0 out of 10,0.969499,0.980319,Not Available,Not Available,Not Available,Not Available,Not Available,0.025396,0.018159,0.027836,Not Available,Not Available,Not Available,Not Available
4,2024,10011,ST. VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,JEFFERSON,0.869247,0.887868,0.859425,0.868485,0 out of 10,3 out of 9,3 out of 10,0.882308,0.907733,0.872458,0.858843,0 out of 10,0 out of 9,0 out of 10,0.840281,0.872976,0.830560,0.782196,0 out of 10,0 out of 9,0 out of 10,0.916491,0.934002,0.905170,0.894272,0 out of 10,0 out of 9,0 out of 10,0.969499,0.980319,0.962992,0.973731,4 out of 10,6 out of 9,6 out of 10,0.025396,0.018159,0.026092,0.026132,0 out of 10,0 out of 9,0 out of 10


In [41]:
# Create list of columns to keep
columns_to_keep_2 = [
    'Facility ID',
    'Facility Name',
    'Address',
    'City/Town',
    'State',
    'ZIP Code',
    'County/Parish',
    'MORT-30-AMI Benchmark',
    'MORT-30-AMI Performance Rate',
    'MORT-30-HF Benchmark',
    'MORT-30-HF Performance Rate',
    'MORT-30-PN Benchmark',
    'MORT-30-PN Performance Rate',
    'MORT-30-COPD Benchmark',
    'MORT-30-COPD Performance Rate'
]

# Drop all remaining columns
df_hvbp_copy = df_hvbp_copy.drop(columns=df_hvbp_copy.columns.difference(columns_to_keep_2))

In [42]:
# Check outpput

df_hvbp_copy.head(1)

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,0.887868,0.880237,0.907733,0.901917,0.872976,0.815392,0.934002,0.909860


> **Addressing Missing Values:**
> 
> During the data profiling phase, **verified the absence of missing values.**

In [43]:
# Check for Null values

df_hvbp_copy.eq('Not Available').sum()

Facility ID                        0
Facility Name                      0
Address                            0
City/Town                          0
State                              0
ZIP Code                           0
County/Parish                      0
MORT-30-AMI Benchmark              0
MORT-30-AMI Performance Rate     721
MORT-30-HF Benchmark               0
MORT-30-HF Performance Rate       80
MORT-30-PN Benchmark               0
MORT-30-PN Performance Rate       63
MORT-30-COPD Benchmark             0
MORT-30-COPD Performance Rate    362
dtype: int64

In [44]:
# Check data types -- To calculate the mean at a later step, quantitative variable columns should be converted to numeric data types

df_hvbp_copy.dtypes

Facility ID                        int64
Facility Name                     object
Address                           object
City/Town                         object
State                             object
ZIP Code                           int64
County/Parish                     object
MORT-30-AMI Benchmark            float64
MORT-30-AMI Performance Rate      object
MORT-30-HF Benchmark             float64
MORT-30-HF Performance Rate       object
MORT-30-PN Benchmark             float64
MORT-30-PN Performance Rate       object
MORT-30-COPD Benchmark           float64
MORT-30-COPD Performance Rate     object
dtype: object

In [45]:
# Convert 'Not Available' values to NaN -- To calculate the mean of each column
df_hvbp_copy.replace('Not Available', np.nan, inplace=True)

# Create list of columns for which data types we want to convert
columns_to_convert_3 = [
    'MORT-30-AMI Performance Rate',
    'MORT-30-HF Performance Rate',
    'MORT-30-PN Performance Rate',
    'MORT-30-COPD Performance Rate'
]

# Convert columns to numeric
df_hvbp_copy[columns_to_convert_3] = df_hvbp_copy[columns_to_convert_3].apply(pd.to_numeric, errors='ignore', downcast='float').astype('float64')

In [46]:
# Check datatypes

df_hvbp_copy.dtypes

Facility ID                        int64
Facility Name                     object
Address                           object
City/Town                         object
State                             object
ZIP Code                           int64
County/Parish                     object
MORT-30-AMI Benchmark            float64
MORT-30-AMI Performance Rate     float64
MORT-30-HF Benchmark             float64
MORT-30-HF Performance Rate      float64
MORT-30-PN Benchmark             float64
MORT-30-PN Performance Rate      float64
MORT-30-COPD Benchmark           float64
MORT-30-COPD Performance Rate    float64
dtype: object

In [47]:
# Create a list of columns for which the mean will be calculated
numeric_columns_w_nans = [
    'MORT-30-AMI Performance Rate',
    'MORT-30-HF Performance Rate',
    'MORT-30-PN Performance Rate',
    'MORT-30-COPD Performance Rate'
]

# Calculate the mean of each column
column_means_3 = df_hvbp_copy[numeric_columns_w_nans].mean()

# Imputate missing values in columns with mean value
df_hvbp_copy.fillna(column_means_3, inplace=True)

In [48]:
# Check output -- Check for null values

df_hvbp_copy.isnull().sum()

Facility ID                      0
Facility Name                    0
Address                          0
City/Town                        0
State                            0
ZIP Code                         0
County/Parish                    0
MORT-30-AMI Benchmark            0
MORT-30-AMI Performance Rate     0
MORT-30-HF Benchmark             0
MORT-30-HF Performance Rate      0
MORT-30-PN Benchmark             0
MORT-30-PN Performance Rate      0
MORT-30-COPD Benchmark           0
MORT-30-COPD Performance Rate    0
dtype: int64

In [49]:
# Check for mixed data types

for col in df_hvbp_copy.columns.tolist():
    weird = (df_hvbp_copy[[col]].map(type) != df_hvbp_copy[[col]].iloc[0].apply(type)).any(axis=1)
    if len (df_hvbp_copy[weird]) > 0:
        print (col)

> Absence of output means there are **no mixed data types present in df_hvbp_copy.**

In [50]:
# Check shape

df_hvbp_copy.shape

(2474, 15)

## 04a. Deriving New Variables - HVBP Clinical Outcomes Dataset

In [51]:
# Deriving new variables for performance indices

df_hvbp_copy['AMI Normative Performance Index'] = df_hvbp_copy['MORT-30-AMI Performance Rate'] / df_hvbp_copy['MORT-30-AMI Benchmark']
df_hvbp_copy['HF Normative Performance Index'] = df_hvbp_copy['MORT-30-HF Performance Rate'] / df_hvbp_copy['MORT-30-HF Benchmark']
df_hvbp_copy['PN Normative Performance Index'] = df_hvbp_copy['MORT-30-PN Performance Rate'] / df_hvbp_copy['MORT-30-PN Benchmark']
df_hvbp_copy['COPD Normative Performance Index'] = df_hvbp_copy['MORT-30-COPD Performance Rate'] / df_hvbp_copy['MORT-30-COPD Benchmark']

> **Deriving these new variables quantifies how hospitals compare numerically with their peers.** Values above 1 indicate that a hospital is outperforming its peers.

In [52]:
#Create threshold
threshold_3 = 1.0

# Derive new variables for comparison flags
df_hvbp_copy['AMI Performance Comparison'] = df_hvbp_copy['AMI Normative Performance Index'].apply(lambda x: 'Outperforms Peers' if x > threshold_3 else 'Meets or Falls Below Peers')
df_hvbp_copy['HF Performance Comparison'] = df_hvbp_copy['HF Normative Performance Index'].apply(lambda x: 'Outperforms Peers' if x > threshold_3 else 'Meets or Falls Below Peers')
df_hvbp_copy['PN Performance Comparison'] = df_hvbp_copy['PN Normative Performance Index'].apply(lambda x: 'Outperforms Peers' if x > threshold_3 else 'Meets or Falls Below Peers')
df_hvbp_copy['COPD Performance Comparison'] = df_hvbp_copy['COPD Normative Performance Index'].apply(lambda x: 'Outperforms Peers' if x > threshold_3 else 'Meets or Falls Below Peers')

> Creating a **qualitative flag to easily denote whether the hospital outperforms peers, meets peers, or falls below peers.**

In [53]:
# Check output

df_hvbp_copy.head(1)

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,AMI Normative Performance Index,HF Normative Performance Index,PN Normative Performance Index,COPD Normative Performance Index,AMI Performance Comparison,HF Performance Comparison,PN Performance Comparison,COPD Performance Comparison
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,0.887868,0.880237,0.907733,0.901917,0.872976,0.815392,0.934002,0.90986,0.991405,0.993593,0.934037,0.974152,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers


In [54]:
# Check shape

df_hvbp_copy.shape

(2474, 23)

# 05. Data Merging

In [55]:
# Create list of key columns to merge df_hgi_copy and df_hvbp_copy
merge_columns_2_3 = ['Facility Name', 'Address', 'City/Town', 'State', 'ZIP Code', 'County/Parish']

# Merge df_hgi_copy and df_hvbp
df_hgi_hvbp_merged = df_hgi_copy.merge(df_hvbp_copy, on=merge_columns_2_3, indicator=True)

In [56]:
# Check output

df_hgi_hvbp_merged.head(1)

,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,Facility ID,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,AMI Normative Performance Index,HF Normative Performance Index,PN Normative Performance Index,COPD Normative Performance Index,AMI Performance Comparison,HF Performance Comparison,PN Performance Comparison,COPD Performance Comparison,_merge
0,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,10001,0.887868,0.880237,0.907733,0.901917,0.872976,0.815392,0.934002,0.90986,0.991405,0.993593,0.934037,0.974152,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,both


In [57]:
# Check _merge flag

df_hgi_hvbp_merged['_merge'].value_counts(dropna=False)

_merge
both          2301
left_only        0
right_only       0
Name: count, dtype: int64

In [58]:
# Remove _merge flag

df_hgi_hvbp_merged_copy = df_hgi_hvbp_merged.drop(columns='_merge')

In [59]:
# Check output

df_hgi_hvbp_merged_copy.head(1)

,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,Facility ID,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,AMI Normative Performance Index,HF Normative Performance Index,PN Normative Performance Index,COPD Normative Performance Index,AMI Performance Comparison,HF Performance Comparison,PN Performance Comparison,COPD Performance Comparison
0,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,10001,0.887868,0.880237,0.907733,0.901917,0.872976,0.815392,0.934002,0.90986,0.991405,0.993593,0.934037,0.974152,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers


In [60]:
# Check shape

df_hgi_hvbp_merged_copy.shape

(2301, 39)

In [61]:
# Create list of key columns to merge df_hgi_hvbp_merged_copy and df_hrrp_copy
merge_columns_1_2_3 = ['Facility Name', 'Facility ID', 'State']

# Merge df_hgi_hvbp_merged_copy and df_hrrp
df_hrrp_hgi_hvbp_merged = df_hrrp_copy.merge(df_hgi_hvbp_merged_copy, on=merge_columns_1_2_3, indicator=True)

In [62]:
# Check output

df_hrrp_hgi_hvbp_merged.head(1)

,Facility Name,Facility ID,State,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold,Address,City/Town,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,AMI Normative Performance Index,HF Normative Performance Index,PN Normative Performance Index,COPD Normative Performance Index,AMI Performance Comparison,HF Performance Comparison,PN Performance Comparison,COPD Performance Comparison,_merge
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,0.8916,3.5325,3.9618,Below Threshold,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.907733,0.901917,0.872976,0.815392,0.934002,0.90986,0.991405,0.993593,0.934037,0.974152,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,both


In [63]:
# Check _merge flag

df_hrrp_hgi_hvbp_merged['_merge'].value_counts(dropna=False)

_merge
both          10344
left_only         0
right_only        0
Name: count, dtype: int64

In [64]:
# Remove _merge flag

df_hrrp_hgi_hvbp_merged_copy = df_hrrp_hgi_hvbp_merged.drop(columns='_merge')

In [65]:
# Check output

df_hrrp_hgi_hvbp_merged_copy.head(1)

,Facility Name,Facility ID,State,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold,Address,City/Town,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,AMI Normative Performance Index,HF Normative Performance Index,PN Normative Performance Index,COPD Normative Performance Index,AMI Performance Comparison,HF Performance Comparison,PN Performance Comparison,COPD Performance Comparison
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,0.8916,3.5325,3.9618,Below Threshold,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.907733,0.901917,0.872976,0.815392,0.934002,0.90986,0.991405,0.993593,0.934037,0.974152,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers,Meets or Falls Below Peers


In [66]:
# Check Shape

df_hrrp_hgi_hvbp_merged_copy.shape

(10344, 44)

# 06. Consistency Checks on Integreated Dataset

In [67]:
# Create list for reordering column names
column_name_order = [
    'Facility ID',
    'Facility Name',
    'State',
    'Address',
    'City/Town',
    'ZIP Code',
    'County/Parish',
    'Telephone Number',
    'Hospital Type',
    'Hospital Ownership',
    'Emergency Services',
    'Promotes Interoperability of EHRs',
    'Birthing Friendly Facility',
    'Hospital Overall Rating',
    'Measure Name',	
    'Excess Readmission Ratio', 	
    'Observed Readmission Rate', 	
    'Expected Readmission Rate', 	
    'Excess Readmissions Threshold',
    'Count of Facility MORT Measures',
    'MORT Measures Better',
    'MORT Measures Worse',
    'Count of Facility Safety Measures', 	
    'Safety Measures Better', 	
    'Safety Measures Worse', 	
    'Count of Facility READM Measures', 	
    'READM Measures Better', 	
    'READM Measures Worse',
    'MORT-30-AMI Benchmark', 	
    'MORT-30-AMI Performance Rate', 
    'AMI Normative Performance Index', 
    'AMI Performance Comparison',
    'MORT-30-HF Benchmark', 	
    'MORT-30-HF Performance Rate', 
    'HF Normative Performance Index', 
    'HF Performance Comparison',
    'MORT-30-PN Benchmark', 	
    'MORT-30-PN Performance Rate', 
    'PN Normative Performance Index', 
    'PN Performance Comparison',
    'MORT-30-COPD Benchmark', 	
    'MORT-30-COPD Performance Rate', 
    'COPD Normative Performance Index', 
    'COPD Performance Comparison'
]

In [68]:
# Rearrange column names

df_hrrp_hgi_hvbp_merged_copy = df_hrrp_hgi_hvbp_merged_copy[column_name_order]

In [69]:
# Check output

df_hrrp_hgi_hvbp_merged_copy.head(1)

,Facility ID,Facility Name,State,Address,City/Town,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,AMI Normative Performance Index,AMI Performance Comparison,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,HF Normative Performance Index,HF Performance Comparison,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,PN Normative Performance Index,PN Performance Comparison,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,COPD Normative Performance Index,COPD Performance Comparison
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-HIP-KNEE-HRRP,0.8916,3.5325,3.9618,Below Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.90986,0.974152,Meets or Falls Below Peers


In [70]:
# Check shape -- To ensure all colmuns are included

df_hrrp_hgi_hvbp_merged_copy.shape

(10344, 44)

In [71]:
# Check for null values

df_hrrp_hgi_hvbp_merged_copy.isnull().sum()

Facility ID                          0
Facility Name                        0
State                                0
Address                              0
City/Town                            0
ZIP Code                             0
County/Parish                        0
Telephone Number                     0
Hospital Type                        0
Hospital Ownership                   0
Emergency Services                   0
Promotes Interoperability of EHRs    0
Birthing Friendly Facility           0
Hospital Overall Rating              0
Measure Name                         0
Excess Readmission Ratio             0
Observed Readmission Rate            0
Expected Readmission Rate            0
Excess Readmissions Threshold        0
Count of Facility MORT Measures      0
MORT Measures Better                 0
MORT Measures Worse                  0
Count of Facility Safety Measures    0
Safety Measures Better               0
Safety Measures Worse                0
Count of Facility READM M

In [72]:
# Check for mixed data types

for col in df_hrrp_hgi_hvbp_merged_copy.columns.tolist():
    weird = (df_hrrp_hgi_hvbp_merged[[col]].map(type) != df_hrrp_hgi_hvbp_merged[[col]].iloc[0].apply(type)).any(axis=1)
    if len (df_hrrp_hgi_hvbp_merged[weird]) > 0:
        print (col)

> **Absence of mixed data types in df_hrrp_hgi_hvbp_merged_copy** results in no output.

In [73]:
# Check for duplicates

df_dups = df_hrrp_hgi_hvbp_merged_copy[df_hrrp_hgi_hvbp_merged_copy.duplicated()]
df_dups

,Facility ID,Facility Name,State,Address,City/Town,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,AMI Normative Performance Index,AMI Performance Comparison,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,HF Normative Performance Index,HF Performance Comparison,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,PN Normative Performance Index,PN Performance Comparison,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,COPD Normative Performance Index,COPD Performance Comparison


> Absence of row contents means **there are no rows of duplicated data**.

In [74]:
# Check integrated dataframe

df_hrrp_hgi_hvbp_merged_copy.head(10)

,Facility ID,Facility Name,State,Address,City/Town,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,AMI Normative Performance Index,AMI Performance Comparison,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,HF Normative Performance Index,HF Performance Comparison,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,PN Normative Performance Index,PN Performance Comparison,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,COPD Normative Performance Index,COPD Performance Comparison
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-HIP-KNEE-HRRP,0.8916,3.5325,3.9618,Below Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.909860,0.974152,Meets or Falls Below Peers
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-HF-HRRP,1.1003,23.1263,21.0184,Above Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.909860,0.974152,Meets or Falls Below Peers
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-AMI-HRRP,0.9332,12.9044,13.8283,Below Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.909860,0.974152,Meets or Falls Below Peers
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-PN-HRRP,0.9871,17.0529,17.2762,Below Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.909860,0.974152,Meets or Falls Below Peers
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-CABG-HRRP,0.9517,9.8131,10.3112,Below Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.909860,0.974152,Meets or Falls Below Peers
5,10001,SOUTHEAST HEALTH MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,DOTHAN,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,READM-30-COPD-HRRP,0.9803,18.1837,18.5483,Below Threshold,7.0,1.0,1.0,8.0,2.0,0.0,11.0,0.0,1.0,0.887868,0.880237,0.991405,Meets or Falls Below Peers,0.907733,0.901917,0.993593,Meets or Falls Below Peers,0.872976,0.815392,0.934037,Meets or Falls Below Peers,0.934002,0.909860,0.974152,Meets or Falls Below Peers
6,10005,MARSHALL MEDICAL CENTERS,AL,2505 U S HIGHWAY 431 NORTH,B

In [75]:
# Find summary statitics of merged datset

df_hrrp_hgi_hvbp_merged_copy.describe(include='all')

,Facility ID,Facility Name,State,Address,City/Town,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,Emergency Services,Promotes Interoperability of EHRs,Birthing Friendly Facility,Hospital Overall Rating,Measure Name,Excess Readmission Ratio,Observed Readmission Rate,Expected Readmission Rate,Excess Readmissions Threshold,Count of Facility MORT Measures,MORT Measures Better,MORT Measures Worse,Count of Facility Safety Measures,Safety Measures Better,Safety Measures Worse,Count of Facility READM Measures,READM Measures Better,READM Measures Worse,MORT-30-AMI Benchmark,MORT-30-AMI Performance Rate,AMI Normative Performance Index,AMI Performance Comparison,MORT-30-HF Benchmark,MORT-30-HF Performance Rate,HF Normative Performance Index,HF Performance Comparison,MORT-30-PN Benchmark,MORT-30-PN Performance Rate,PN Normative Performance Index,PN Performance Comparison,MORT-30-COPD Benchmark,MORT-30-COPD Performance Rate,COPD Normative Performance Index,COPD Performance Comparison
count,10344.000000,10344,10344,10344,10344,10344.000000,10344,10344,10344,10344,10344,10344,10344,10344,10344,10344.000000,10344.000000,10344.000000,10344,10344.000000,10344.000000,10344.000000,10344.000000,10344.000000,10344.000000,10344.000000,10344.000000,10344.000000,1.034400e+04,10344.000000,10344.000000,10344,1.034400e+04,10344.000000,10344.000000,10344,1.034400e+04,10344.000000,10344.000000,10344,1.034400e+04,10344.000000,10344.000000,10344
unique,NaN,2221,49,2259,1471,NaN,817,2259,1,10,2,2,2,5,6,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,2
top,NaN,MERCY MEDICAL CENTER,CA,1 MEDICAL CENTER DRIVE,CHICAGO,NaN,LOS ANGELES,(952) 892-2101,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Y,Y,3,READM-30-PN-HRRP,NaN,NaN,NaN,Below Threshold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Meets or Falls Below Peers,NaN,NaN,NaN,Meets or Falls Below Peers,NaN,NaN,NaN,Meets or Falls Below Peers,NaN,NaN,NaN,Meets or Falls Below Peers
freq,NaN,21,942,17,72,NaN,239,23,10344,5360,9986,9841,8207,2925,2241,NaN,NaN,NaN,5284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9533,NaN,NaN,NaN,9926,NaN,NaN,NaN,10286,NaN,NaN,NaN,10324
mean,255746.089907,NaN,NaN,NaN,NaN,51012.222061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.002792,15.164002,15.124622,NaN,5.775329,0.236659,0.174207,6.429911,1.111176,0.343194,9.142885,0.417923,0.915507,8.878680e-01,0.874252,0.984665,NaN,9.077330e-01,0.879884,0.969320,NaN,8.729760e-01,0.815171,0.933784,NaN,9.340020e-01,0.904835,0.968773,NaN
std,157604.085506,NaN,NaN,NaN,NaN,28088.904883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076837,5.434114,5.308432,NaN,1.378481,0.628581,0.490663,1.651007,1.007856,0.615917,1.754948,0.718959,1.130392,1.515528e-13,0.010203,0.011492,NaN,7.216799e-15,0.016746,0.018448,NaN,6.894818e-14,0.024457,0.028016,NaN,3.353036e-14,0.012421,0.013298,NaN
min,10001.000000,NaN,NaN,NaN,NaN,1040.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615900,2.611200,2.830700,NaN,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,8.878680e-01,0.830732,0.935648,NaN,9.077330e-01,0.807235,0.889287,NaN,8.729760e-01,0.712037,0.815643,NaN,9.340020e-01,0.841646,0.901118,NaN
25%,110002.750000,NaN,NaN,NaN,NaN,30077.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.957900,12.457175,12.501775,NaN,5.000000,0.000000,0.000000,6.000000,0.000000,0.000000,8.000000,0.000000,0.000000,8.878680e-01,0.869248,0.979029,NaN,9.077330e-01,0.869277,0.957635,NaN,8.729760e-01,0.800638,0.917137,NaN,9.340020e-01,0.898410,0.961893,NaN
50%,250040.000000,NaN,NaN,NaN,NaN,48060.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998400,16.535250,16.627400,NaN,6.000000,0.000000,0.000000,7.000000,1.000000,0.000000,9.000000,0.000000,1.000000,8.878680e-01,0.874233,0.984643,NaN,9.077330e-01,0.880347,0.969830,NaN,8.729760e-01,0.816225,0.934991,NaN,9.340020e-01,0.904893,0.968834,NaN
75%,390009.750000,NaN,NaN,NaN,NaN,76015.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.043325,19.094100,19.170

In [76]:
# Check shape of integrated dataframe

df_hrrp_hgi_hvbp_merged_copy.shape

(10344, 44)

# 07. Exporting Data

In [77]:
# Export cleaned Hospital Readmissions Reduction Program dataset
df_hrrp_copy.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hospital_readmissions_reduction_program_cleaned.csv'), index=False)

In [78]:
# Export cleaned Hospital General information dataset
df_hgi_copy.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hospital_general_information_cleaned.csv'), index=False)

In [79]:
# Export cleaned HVBP Clinical Outcomes datset
df_hvbp_copy.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hvbp_clinical_outcomes_cleaned.csv'), index=False)

In [80]:
# Export cleaned and merged HGI and HVBP Clinical Outcomes dataset
df_hgi_hvbp_merged_copy.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hgi_hvbp_merged.csv'), index=False)

In [81]:
# Export cleaned and integrated HRRP, HGI, and HVBP Clinical Outcomes dataset
df_hrrp_hgi_hvbp_merged_copy.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hrrp_hgi_hvbp_merged.csv'), index=False)